In [ ]:
import pandas as pd
from pandas import DataFrame
import requests
from bs4 import BeautifulSoup

In [ ]:
# 수집할 사이트 주소 설정
params = ['', 'day', 'week', 'month']

# dict 형식의 노래제목, 가수, 앨범, 순위를 담을 빈 list 생성
# 실시간, 일, 주, 월별 순위를 담을 list를 생성
melon_chart = []
melon_chart_day = []
melon_chart_week = []
melon_chart_month = []


# 실시간, 일, 주, 월별 데이터를 저장할 list 생성
chart_list = [melon_chart, melon_chart_day, melon_chart_week, melon_chart_month]

for i, param in enumerate(params):
    # 수집할 컨텐츠가 있는 웹 페이지 주소
    url = f'https://www.melon.com/chart/{param}/index.htm'

    # 브라우저 버전정보
    userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

    # 접속 객체 생성
    # request는 직접 header 정보 갱신이 불가능해 session 생성
    session = requests.Session()

    # 접속 객체에 부가정보(header) 삽입
    session.headers.update({
        "Referer": "",
        "User-Agent": userAgent
    })

    # 생성한 접속 객체를 활용해 웹 페이지에 접속
    # 지정된 URL에 GET 요청을 보내고
    # 그에 대한 응답을 r 변수에 할당
    r = session.get(url)

    # 접속 실패 시 에러 코드와 에러 메시지 출력
    if r.status_code != 200:
        msg = "[%d Error] %s 에러 발생" % (r.status_code, r.reason)
        raise Exception(msg)

    # 인코딩 형식을 지정해 beautifulsoup 객체를 생성
    r.encoding = 'utf-8'

    # r 변수에 저장된 text 정보를 시각화, 확인
    # print(r.text)

    # r.text에서 HTML을 파싱하고,
    # 파싱된 HTML을 나타내는
    # BeautifulSoup 객체 soup를 생성
    soup = BeautifulSoup(r.text)

    # 추출할 음원의 수를 확인하기 위해 음악 이름을 나타내는
    # .ellipsis.rank01의 수를 확인
    melon_list = soup.select(".ellipsis.rank01")

    for j in range(len(melon_list)):
        # 노래 추출
        songEl = soup.select('.ellipsis.rank01 > span > a')
        song = songEl[j].text.strip()

        # 가수 추출
        singerEl = soup.select('.ellipsis.rank02 > a')
        singer = singerEl[j].text.strip()

        # 앨범 추출
        albumEl = soup.select('.ellipsis.rank03 > a')
        album = albumEl[j].text.strip()

        # 순위
        rank = j + 1

        item = {"가수": singer, "노래 제목": song, "앨범": album, "순위": rank}
        chart_list[i].append(item)
chart_list

In [6]:
# 생성된 각각의 chart별로 정리를 해줘야함
df_melon_chart = pd.DataFrame(melon_chart)
df_melon_chart_day = pd.DataFrame(melon_chart_day)
df_melon_chart_week = pd.DataFrame(melon_chart_week)
df_melon_chart_month = pd.DataFrame(melon_chart_month)


In [7]:
# 순위를 우선해서 보여주기 위해 index로 설정
df_melon_chart.set_index('순위', inplace=True)
df_melon_chart_day.set_index('순위', inplace=True)
df_melon_chart_week.set_index('순위', inplace=True)
df_melon_chart_month.set_index('순위', inplace=True)

In [8]:
# 각각의 데이터를 별도의 xlsx 파일로 저장
df_melon_chart.to_excel('확인용_100.xlsx')
df_melon_chart_day.to_excel('확인용_day.xlsx')
df_melon_chart_week.to_excel('확인용_week.xlsx')
df_melon_chart_month.to_excel('확인용_month.xlsx')